In [2]:
import math
import numpy as np
import pandas as pd
import geopandas as gpd
from pandas import testing as tm
from ortools.sat.python import cp_model

In [3]:
df = pd.read_csv("fountains-distances-join-fountains-counts.csv")
df.head(4)

,id,nearest_rev,nearest_fountain,nearest_rvertes,nearest_pcyclable,nearest_murbain,nearest_iexterne,min_col,min_val,longitude,latitude,objectid,superficie,count
0,1,2697.569104,48.907970,2512.295088,176.113692,2529.702004,21,nearest_iexterne,21.446787,-73.589462,45.592012,5069,11.80462,5
1,2,2745.451551,1.408827,2541.866572,167.330400,2495.967421,33,nearest_fountain,1.408827,-73.590055,45.592189,5069,11.80462,5
2,3,2744.043719,1.408827,2540.756133,167.190902,2497.152536,32,nearest_fountain,1.408827,-73.590041,45.592181,5069,11.80462,5
3,4,2582.406191,141.348494,2371.053545,84.592035,2670.529930,18,nearest_iexterne,18.837126,-73.589008,45.590781,5069,11.80462,5


In [4]:
print(f"length < 1 (ha): {len(df[df['superficie'] < 1])}")
print(f"length 1-5 (ha): {len(df[(df['superficie'] >= 1) & (df['superficie'] < 5)])}")
print(f"length 5-20 (ha): {len(df[(df['superficie'] >= 5) & (df['superficie'] < 20)])}")
print(f"length 20-100 (ha): {len(df[(df['superficie'] >= 20) & (df['superficie'] < 100)])}")
print(f"length > 100 (ha): {len(df[df['superficie'] > 100])}")

length < 1 (ha): 230
length 1-5 (ha): 299
length 5-20 (ha): 197
length 20-100 (ha): 66
length > 100 (ha): 31


In [5]:
bins = [0, 1, 5, 20, 100, float('inf')]
labels = ['< 1', '1-5', '5-20', '20-100', '> 100']

# Ajout d'une colonne 'categorie' au DataFrame pour stocker la catégorie de chaque superficie
df['categorie'] = pd.cut(df['superficie'], bins=bins, labels=labels)

df.head(4)

,id,nearest_rev,nearest_fountain,nearest_rvertes,nearest_pcyclable,nearest_murbain,nearest_iexterne,min_col,min_val,longitude,latitude,objectid,superficie,count,categorie
0,1,2697.569104,48.907970,2512.295088,176.113692,2529.702004,21,nearest_iexterne,21.446787,-73.589462,45.592012,5069,11.80462,5,5-20
1,2,2745.451551,1.408827,2541.866572,167.330400,2495.967421,33,nearest_fountain,1.408827,-73.590055,45.592189,5069,11.80462,5,5-20
2,3,2744.043719,1.408827,2540.756133,167.190902,2497.152536,32,nearest_fountain,1.408827,-73.590041,45.592181,5069,11.80462,5,5-20
3,4,2582.406191,141.348494,2371.053545,84.592035,2670.529930,18,nearest_iexterne,18.837126,-73.589008,45.590781,5069,11.80462,5,5-20


In [6]:
dictionnary = df.groupby("categorie").agg({"count": "max"}).to_dict()["count"]
df['max-fountain'] = df['categorie'].map(dictionnary)
df = df.rename(columns={"count": "nbr-fountain"})

df.head(4)

,id,nearest_rev,nearest_fountain,nearest_rvertes,nearest_pcyclable,nearest_murbain,nearest_iexterne,min_col,min_val,longitude,latitude,objectid,superficie,nbr-fountain,categorie,max-fountain
0,1,2697.569104,48.907970,2512.295088,176.113692,2529.702004,21,nearest_iexterne,21.446787,-73.589462,45.592012,5069,11.80462,5,5-20,11
1,2,2745.451551,1.408827,2541.866572,167.330400,2495.967421,33,nearest_fountain,1.408827,-73.590055,45.592189,5069,11.80462,5,5-20,11
2,3,2744.043719,1.408827,2540.756133,167.190902,2497.152536,32,nearest_fountain,1.408827,-73.590041,45.592181,5069,11.80462,5,5-20,11
3,4,2582.406191,141.348494,2371.053545,84.592035,2670.529930,18,nearest_iexterne,18.837126,-73.589008,45.590781,5069,11.80462,5,5-20,11


In [7]:
# Conversion en mètres carrés
df['superficie-metre'] = df['superficie'] * 10000

df.head(4)

,id,nearest_rev,nearest_fountain,nearest_rvertes,nearest_pcyclable,nearest_murbain,nearest_iexterne,min_col,min_val,longitude,latitude,objectid,superficie,nbr-fountain,categorie,max-fountain,superficie-metre
0,1,2697.569104,48.907970,2512.295088,176.113692,2529.702004,21,nearest_iexterne,21.446787,-73.589462,45.592012,5069,11.80462,5,5-20,11,118046.202996
1,2,2745.451551,1.408827,2541.866572,167.330400,2495.967421,33,nearest_fountain,1.408827,-73.590055,45.592189,5069,11.80462,5,5-20,11,118046.202996
2,3,2744.043719,1.408827,2540.756133,167.190902,2497.152536,32,nearest_fountain,1.408827,-73.590041,45.592181,5069,11.80462,5,5-20,11,118046.202996
3,4,2582.406191,141.348494,2371.053545,84.592035,2670.529930,18,nearest_iexterne,18.837126,-73.589008,45.590781,5069,11.80462,5,5-20,11,118046.202996


In [8]:
grouped = df.groupby('objectid')\
            .agg({
                'superficie-metre': 'first', 
                'nbr-fountain': 'first', 
                'max-fountain': 'first'})\
                    .reset_index()

print(grouped)

     objectid  superficie-metre  nbr-fountain max-fountain
0        3479      5.426809e+03             1            3
1        3482      4.711487e+04             4            5
2        3485      3.513201e+04             2            5
3        3494      4.019093e+04             2            5
4        3523      2.378247e+03             1            3
..        ...               ...           ...          ...
477      5659      1.775433e+05             2           11
478      5662      6.249648e+05             3           13
479      5679      1.278951e+04             2            5
480      5685      2.541568e+06             5           18
481      5691      1.546228e+05             1           11

[482 rows x 4 columns]


In [9]:
grouped['max-fountain'] = grouped['max-fountain'].astype(int)

# Group by id_parc and get the max value of nbr_fountain for each group
max_fountains = grouped.groupby('objectid')['max-fountain'].max()

# Sum the maximum values
sum_max_fountains = max_fountains.sum()

# Print the result
print(sum_max_fountains)

2525


In [10]:
# Create the CP-SAT model
model = cp_model.CpModel()

# Define the variables
fountains = [model.NewIntVar(0, grouped['max-fountain'][i], f'fountains{i}') for i in range(len(grouped))]

# Define the constraints
for i in range(len(grouped)):
    # Contrainte pour garantir qu'il y a au moins une fontaine par parc
    model.Add(fountains[i] >= 1)
    # Contrainte pour limiter le nombre de fontaines par parc
    model.Add(fountains[i] <= grouped['max-fountain'][i])
    # Contrainte pour limiter le nombre de fontaines en fonction de la superficie du parc
    model.Add(fountains[i] <= math.ceil(grouped['superficie-metre'][i] // 100))
    
# Contrainte pour limiter le nombre total de fontaines
model.Add(sum(fountains) <= sum_max_fountains)

# Define the objective function
model.Maximize(sum(fountains))

# Create the solver and solve the model
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Print the solution
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print('Solution trouvée avec un nombre total de fontaines de', solver.ObjectiveValue(), ':')
    for i in range(len(grouped)):
        print(f'Parc {grouped["objectid"][i]}: {solver.Value(fountains[i])} fontaines')
else:
    print('Aucune solution trouvée')

Solution trouvée avec un nombre total de fontaines de 2525.0 :
Parc 3479: 3 fontaines
Parc 3482: 5 fontaines
Parc 3485: 5 fontaines
Parc 3494: 5 fontaines
Parc 3523: 3 fontaines
Parc 3524: 11 fontaines
Parc 3526: 5 fontaines
Parc 3528: 5 fontaines
Parc 3531: 3 fontaines
Parc 3538: 3 fontaines
Parc 3548: 5 fontaines
Parc 3551: 5 fontaines
Parc 3557: 5 fontaines
Parc 3563: 5 fontaines
Parc 3567: 5 fontaines
Parc 3568: 3 fontaines
Parc 3570: 3 fontaines
Parc 3581: 5 fontaines
Parc 3589: 11 fontaines
Parc 3592: 5 fontaines
Parc 3602: 3 fontaines
Parc 3617: 3 fontaines
Parc 3622: 3 fontaines
Parc 3624: 3 fontaines
Parc 3631: 11 fontaines
Parc 3639: 5 fontaines
Parc 3642: 11 fontaines
Parc 3647: 5 fontaines
Parc 3651: 11 fontaines
Parc 3657: 3 fontaines
Parc 3660: 5 fontaines
Parc 3667: 5 fontaines
Parc 3672: 5 fontaines
Parc 3674: 5 fontaines
Parc 3678: 11 fontaines
Parc 3680: 5 fontaines
Parc 3684: 3 fontaines
Parc 3692: 5 fontaines
Parc 3700: 18 fontaines
Parc 3705: 3 fontaines
Parc 3706:

In [11]:
print(str(model))

variables {
  name: "fountains0"
  domain: 0
  domain: 3
}
variables {
  name: "fountains1"
  domain: 0
  domain: 5
}
variables {
  name: "fountains2"
  domain: 0
  domain: 5
}
variables {
  name: "fountains3"
  domain: 0
  domain: 5
}
variables {
  name: "fountains4"
  domain: 0
  domain: 3
}
variables {
  name: "fountains5"
  domain: 0
  domain: 11
}
variables {
  name: "fountains6"
  domain: 0
  domain: 5
}
variables {
  name: "fountains7"
  domain: 0
  domain: 5
}
variables {
  name: "fountains8"
  domain: 0
  domain: 3
}
variables {
  name: "fountains9"
  domain: 0
  domain: 3
}
variables {
  name: "fountains10"
  domain: 0
  domain: 5
}
variables {
  name: "fountains11"
  domain: 0
  domain: 5
}
variables {
  name: "fountains12"
  domain: 0
  domain: 5
}
variables {
  name: "fountains13"
  domain: 0
  domain: 5
}
variables {
  name: "fountains14"
  domain: 0
  domain: 5
}
variables {
  name: "fountains15"
  domain: 0
  domain: 3
}
variables {
  name: "fountains16"
  domain: 0
  d

In [19]:
import re

# the input string
input_str = """
variables {
  name: "fountains480"
  domain: 0
  domain: 18
}
variables {
  name: "fountains481"
  domain: 0
  domain: 11
}
constraints {
  linear {
    vars: 0
    coeffs: 1
    domain: 1
    domain: 9223372036854775807
  }
}
constraints {
  linear {
    vars: 0
    coeffs: 1
    domain: -9223372036854775808
    domain: 3
  }
}
"""

# initialize an empty dictionary to store the extracted information
result_dict = {
    "variables": [],
    "constraints": []
}

# use regex to extract the necessary information and store it in the dictionary
variable_regex = re.compile(r'variables\s*{\s*name:\s*"(.+)"\s*domain:\s*(\d+)\s*domain:\s*(\d+)\s*}\s*')
constraint_regex = re.compile(r'constraints\s*{\s*linear\s*{\s*vars:\s*(\d+)\s*coeffs:\s*(\d+)\s*domain:\s*(.+)\s*domain:\s*(.+)\s*}\s*}\s*')

for match in variable_regex.finditer(input_str):
    result_dict["variables"].append({
        "name": match.group(1),
        "domain": [int(match.group(2)), int(match.group(3))]
    })

for match in constraint_regex.finditer(input_str):
    result_dict["constraints"].append({
        "vars": int(match.group(1)),
        "coeffs": int(match.group(2)),
        "domain": [int(match.group(3)), int(match.group(4))]
    })

print(result_dict)


{'variables': [{'name': 'fountains480', 'domain': [0, 18]}, {'name': 'fountains481', 'domain': [0, 11]}], 'constraints': [{'vars': 0, 'coeffs': 1, 'domain': [1, 9223372036854775807]}, {'vars': 0, 'coeffs': 1, 'domain': [-9223372036854775808, 3]}]}


In [20]:
class SolutionPrinter(cp_model.CpSolverSolutionCallback):
        
    def __init__(self, fountains, results):
        """Fonction de callback qui stocke les solutions trouvées"""
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.fountains = fountains
        self.results = results
        self.solution_count = 0
    
    def OnSolutionCallback(self):
        """Stocke la solution trouvée dans le dictionnaire results"""
        self.solution_count += 1
        for i, f in enumerate(self.fountains):
            self.results[f'Parc {i+1}'] = self.Value(f)


In [21]:
# Create a list of dictionaries to store the solution data
solution_data = []
for i in range(len(grouped)):
    solution_data.append({'objectid': grouped['objectid'][i], 'nbr-fountain': solver.Value(fountains[i])})

# Create a DataFrame from the solution data
solution_df = pd.DataFrame(solution_data)

# Export the DataFrame to a CSV file
# solution_df.to_csv('solution.csv', index=False)

In [22]:
df_ortools = solution_df.copy()
df_ortools.head(10)

,objectid,nbr-fountain
0,3479,3
1,3482,5
2,3485,5
3,3494,5
4,3523,3
5,3524,11
6,3526,5
7,3528,5
8,3531,3
9,3538,3


In [23]:
columns = ["objectid", "nbr-fountain"]
df_origine = grouped.copy()[columns]
df_origine.head(10)

,objectid,nbr-fountain
0,3479,1
1,3482,4
2,3485,2
3,3494,2
4,3523,1
5,3524,1
6,3526,1
7,3528,2
8,3531,1
9,3538,1


In [24]:
# df_origine = grouped.copy()
# df_origine = df_origine.rename(columns={"count": "nbr_fountain"})
# df_origine = df_origine.astype({"objectid": "int64", "nbr_fountain": "int64", "max-fountains": "int64"})

# df_origine.head(4)

In [25]:
# df_ortools = df_ortools.rename(columns={"max-fountains": "nbr_fountain"})
# df_ortools = df_ortools.astype({"objectid": "int64", "nbr_fountain": "int64", "superficie-metre": "float64"})

# df_ortools.head(4)

In [26]:
# # Joindre les deux dataframes sur la colonne 'id_parc'
# df_compare = pd.merge(df_origine, df_ortools, on='objectid', suffixes=('_origine', '_ortools'))

# # Ajouter une colonne 'compare' pour indiquer si le nombre de fontaines généré par ortools est supérieur, égal ou inférieur au nombre d'origine
# df_compare['compare'] = np.where(df_compare['nbr_fountain_ortools'] > df_compare['nbr_fountain_origine'], 'supérieur', np.where(df_compare['nbr_fountain_ortools'] == df_compare['nbr_fountain_origine'], 'égal', 'inférieur'))

# # Afficher le dataframe de comparaison
# df_compare.head(10)
